# Actividad guiada 3

David Ricardo Ortigoza Micolta

colab =https://colab.research.google.com/drive/1Lx9gjXEqVP3tazxz-5zIKTwPCCoYPsu3?usp=sharing

github= 

In [27]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
#librerias
import urllib.request # Libreria para llamar paginas web
import tsplib95       # Modulo para las instancias del problema del TSP
import math           # Libreria para hacer funciones matematicas basada en C
import random         # Modulo para aleatorizar 

## Carga de datos

In [36]:
#descarga de datos

file= "swiss42.tsp" #nombre del archivo
urllib.request.urlretrieve(f"http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz",file+'.gz') # descargue la info de la url
#usamos comando del sistema para descomprimir el archivo
!gzip -d swiss42.tsp.gz

gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? y


In [37]:
# carga de datos 

# uso de tslip para leer datos

problema= tsplib95.load(file) #asigno la información a la variable problema

# definición de los nodos 

nodos,aristas= list(problema.get_nodes()),list(problema.get_edges())


## Funciones Básicas

In [38]:
# Genear solución aleatoria que comience en el nodo 0
def crear_solucion(nodos):
  #definición del nodo 0 en la solución que vamos a crear
  solucion=[nodos[0]]
  #loop para crear solución aleatoria
  for i in nodos[1:]:
    # pega los nodos seleccionados a la cadena y lo elimina de la lista dsiponible para escoger
    solucion=solucion + [random.choice(list(set(nodos) - set({nodos[0]}) - set(solucion)))]
  return(solucion)

In [39]:
# distancia entre nodos
def distancia(a,b,problema):
  return problema.get_weight(a,b)


In [40]:
# calculo de la distancia totla de una solución
def distancia_total(solucion,problema):
  distancia_total=0
  for i in range(len(solucion)-1):
    #cálculo de la distancia entre los nodos seleccionados aleatoriamente
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problema)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problema)

## Heuristica: Busqueda aleatoria

In [49]:
def busqueda_voraz_aleatoria(problema,n):
  Nodos=list(problema.get_nodes())
  #lista para almacener la distancia
  mejor_solucion=[]
  #valor para mejor distancia
  mejor_distancia=float('inf')
  for i in range(n):
    #llamar iterativamente la solución para crear lista de nodos aleatorios
    solucion=crear_solucion(Nodos)
    #calcular la distancia para la solución creada
    distancia=distancia_total(solucion,problema)
    #condicional para evaluar distancia
    if distancia<mejor_distancia:
      mejor_distancia=distancia
      mejor_solucion=solucion
    #imprimir soluciones
  print(f'La mejor solicion es {mejor_solucion}')
  print(f'La distancia para la solución es: {mejor_distancia}')
  return mejor_solucion

In [50]:
# Realizar la busqueda aleatoria con 5000 iteraciones
solucion=busqueda_voraz_aleatoria(problema,10000)

La mejor solicion es [0, 31, 17, 21, 12, 11, 8, 13, 27, 35, 39, 40, 24, 23, 10, 3, 30, 18, 19, 32, 36, 14, 26, 28, 4, 29, 2, 15, 5, 25, 22, 34, 1, 7, 20, 33, 38, 37, 16, 41, 9, 6]
La distancia para la solución es: 3742


## Busqueda Local (trabaja con busqueda en el vecindario)

In [43]:
def generar_vecino(solucion):
  # creamos variables para almacenar la mejor solución y distancia
  mejor_solucion=[]
  mejor_distancia=float('inf')
  # bucle para recorrer los nodos y evaluar vecindad
  for i in range(1,len(solucion)-1):
    for j in range(i+1,len(solucion)):
      # se crea el vecino concatenando listas
      vecino=solucion[:i]+ [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      # evaluamos distancia de la solución
      distancia=distancia_total(vecino,problema)
      # condicional para guardar la mejor solución encontrada
      if distancia<=mejor_distancia:
        mejor_distancia=distancia
        mejor_solucion=vecino
  return mejor_solucion

In [53]:
#generamos una solución con el algoritmo anterior
print("Distancia Solucion Incial:" , distancia_total(solucion, problema))
#creando nueva solución con busqueda local
nueva_solucion = generar_vecino(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problema))

Distancia Solucion Incial: 3742
Distancia Mejor Solucion Local: 3438


In [64]:
def  busqueda_local(problema):
  # creamos variables para almacenar la mejor solución y distancia
  mejor_solucion=[]
  #generamos solución inicial de referencia (aleatoria)
  solucion_referencia=crear_solucion(nodos)
  mejor_distancia=distancia_total(solucion_referencia,problema)
  #iteraciones
  iteracion=0
  while(1):
    #incrementamos el contador
    iteracion+=1
    #creamos un vencindario para la solución
    vecino=generar_vecino(solucion_referencia)
    #caluclamos distancia
    distancia_vecindad=distancia_total(vecino,problema)
    #condicional de evaluación de vecindad
    if distancia_vecindad<mejor_distancia:
      #asigno la mejor solución y distancia
      mejor_solucion=vecino
      mejor_distancia=distancia_vecindad
    else:
      print(f'En la iteracion {iteracion} la mejor solución encontrada es: {mejor_solucion} con una distancia {mejor_distancia}')
      #el return romple el ciclo
      return(mejor_solucion)
    solucion_referencia=vecino

In [65]:
sol = busqueda_local(problema)

En la iteracion 30 la mejor solución encontrada es: [0, 2, 27, 17, 36, 35, 31, 32, 9, 21, 39, 22, 30, 4, 5, 26, 18, 10, 8, 29, 28, 3, 14, 16, 15, 37, 7, 25, 41, 23, 40, 24, 38, 34, 33, 20, 19, 13, 11, 12, 6, 1] con una distancia 2043


## Recocido simulado
Algorítmo basado en la termodinámica

In [66]:
def generar_vecino_aleatorio(solucion):
  # asignar 2 nodos aleatoriamente
  i,j = sorted(random.sample(range(1,len(solucion)),2))
  #retonrar una solucion con los nodos intercambiados
  

In [72]:
#criterio de selección por pobrabilidad
def probabilidad(T,D):
  if random.random()<math.exp(-1*D/T):
    return True
  else:
    return False

In [80]:
def bajar_temp(T):
  return(T*0.99)

In [83]:
def recocido_simulado(problema,T):
  #creo el estado inicial para el algoritmo
  solucion_referencia=crear_solucion(nodos)
  distancia_referencia=distancia_total(solucion_referencia,problema)
  mejor_solucion=[]
  mejor_distancia=10e100
  n=0
  # Bucle para evauar la temperatura
  while T >0.0001:
    n+=1
    #calcular soluciones vecinas
    vecina=generar_vecino(solucion_referencia)
    #calculamos la distancia del vecino
    distancia_vecino=distancia_total(vecina,problema)
    #condicional para hallar la mejor solucion
    if distancia_vecino < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecino
    
    #condicional para evalauar la distancia, depende de si es mejor o por probabilidad
    if distancia_vecino<distancia_referencia or probabilidad(T,abs(distancia_referencia-distancia_vecino)):
      solucion_referencia=vecina
      distancia_referencia=distancia_vecino
    T=bajar_temp(T)
  print(f'La mejor solución es: {mejor_solucion} con una distancia de {mejor_distancia}')

In [84]:
sol  = recocido_simulado(problema, 10000000)

La mejor solución es: [0, 30, 29, 9, 23, 41, 25, 11, 12, 4, 3, 2, 27, 28, 32, 34, 33, 20, 31, 17, 38, 22, 24, 40, 21, 39, 8, 10, 18, 13, 19, 14, 16, 15, 37, 36, 35, 7, 5, 26, 6, 1] con una distancia de 1657
